## **Installing Dependencies**

In [ ]:
!pip install tensorflow

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import warnings
warnings.filterwarnings("ignore")

## **Importing Libraries**

In [ ]:
import pandas as pd
import numpy as np
import ast
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

## **Load the data**

Load the data with the hidden states extracted from the LLM

In [ ]:
path = '/content/drive/MyDrive/Ceccarelli_MasterThesis&Internship/Master Thesis/embeddings_saplma/'

In [ ]:
hallu_factalign_1_0_1000 = pd.read_csv(path + 'factalign/embeddings_factalign_llama3.2-3B_1_query_answer.csv', index_col=0)
hallu_factalign_4_0_1000 = pd.read_csv(path + 'factalign/embeddings_factalign_llama3.2-3B_4_query_answer.csv', index_col=0)
hallu_factalign_8_0_1000 = pd.read_csv(path + 'factalign/embeddings_factalign_llama3.2-3B_8_query_answer.csv', index_col = 0)
hallu_factalign_16_0_1000 = pd.read_csv(path + 'factalign/embeddings_factalign_llama3.2-3B_16_query_answer.csv', index_col = 0)

hallu_factalign_1_1000_end = pd.read_csv(path + 'factalign/embeddings_factalign_llama3.2-3B_1_query_answer_1000:.csv', index_col=0)
hallu_factalign_4_1000_end = pd.read_csv(path + 'factalign/embeddings_factalign_llama3.2-3B_4_query_answer_1000:.csv', index_col=0)
hallu_factalign_8_1000_end = pd.read_csv(path + 'factalign/embeddings_factalign_llama3.2-3B_8_query_answer_1000:.csv', index_col = 0)
hallu_factalign_16_1000_end = pd.read_csv(path + 'factalign/embeddings_factalign_llama3.2-3B_16_query_answer_1000:.csv', index_col = 0)

hallu_factbench_1 = pd.read_csv(path + 'factbench/embeddings_factbench_llama3.2-3B_1_query_answer.csv', index_col=0)
hallu_factbench_4 = pd.read_csv(path + 'factbench/embeddings_factbench_llama3.2-3B_4_query_answer.csv', index_col=0)
hallu_factbench_8 = pd.read_csv(path + 'factbench/embeddings_factbench_llama3.2-3B_8_query_answer.csv', index_col = 0)
hallu_factbench_16 = pd.read_csv(path + 'factbench/embeddings_factbench_llama3.2-3B_16_query_answer.csv', index_col = 0)

hallu_felm_1 = pd.read_csv(path + 'felm/embeddings_felm_llama3.2-3B_1_query_answer.csv', index_col=0)
hallu_felm_4 = pd.read_csv(path + 'felm/embeddings_felm_llama3.2-3B_4_query_answer.csv', index_col=0)
hallu_felm_8 = pd.read_csv(path + 'felm/embeddings_felm_llama3.2-3B_8_query_answer.csv', index_col = 0)
hallu_felm_16 = pd.read_csv(path + 'felm/embeddings_felm_llama3.2-3B_16_query_answer.csv', index_col = 0)

In [ ]:
hallu_factalign_1 = pd.concat([hallu_factalign_1_0_1000, hallu_factalign_1_1000_end]).reset_index()
hallu_factalign_4 = pd.concat([hallu_factalign_4_0_1000, hallu_factalign_4_1000_end]).reset_index()
hallu_factalign_8 = pd.concat([hallu_factalign_8_0_1000, hallu_factalign_8_1000_end]).reset_index()
hallu_factalign_16 = pd.concat([hallu_factalign_16_0_1000, hallu_factalign_16_1000_end]).reset_index()

hallu_factbench_1 = hallu_factbench_1.reset_index()
hallu_factbench_4 = hallu_factbench_4.reset_index()
hallu_factbench_8 = hallu_factbench_8.reset_index()
hallu_factbench_16 = hallu_factbench_16.reset_index()

hallu_felm_1 = hallu_felm_1.reset_index()
hallu_felm_4 = hallu_felm_4.reset_index()
hallu_felm_8 = hallu_felm_8.reset_index()
hallu_felm_16 = hallu_felm_16.reset_index()

## **Data Preparation**

In [ ]:
# Consider only the prompt, the corresponding response, the factuality label and the embeddings

hallu_factalign_1 = hallu_factalign_1[['prompt', 'completion', 'label', 'embeddings']]
hallu_factalign_4 = hallu_factalign_4[['prompt', 'completion', 'label', 'embeddings']]
hallu_factalign_8 = hallu_factalign_8[['prompt', 'completion', 'label', 'embeddings']]
hallu_factalign_16 = hallu_factalign_16[['prompt', 'completion', 'label', 'embeddings']]

hallu_factbench_1 = hallu_factbench_1[['prompt', 'response', 'response_label', 'embeddings']]
hallu_factbench_4 = hallu_factbench_4[['prompt', 'response', 'response_label', 'embeddings']]
hallu_factbench_8 = hallu_factbench_8[['prompt', 'response', 'response_label', 'embeddings']]
hallu_factbench_16 = hallu_factbench_16[['prompt', 'response', 'response_label', 'embeddings']]

hallu_felm_1 = hallu_felm_1[['prompt', 'response', 'hallucination', 'embeddings']]
hallu_felm_4 = hallu_felm_4[['prompt', 'response', 'hallucination', 'embeddings']]
hallu_felm_8 = hallu_felm_8[['prompt', 'response', 'hallucination', 'embeddings']]
hallu_felm_16 = hallu_felm_16[['prompt', 'response', 'hallucination', 'embeddings']]

In [ ]:
hallu_factbench_1

,prompt,response,response_label,embeddings
0,"Generate a poem with length 10-12 lines, based...",The silence of the night \nIs a peaceful sight...,True,"[-1.7470703125, 0.0831298828125, 4.203125, -2...."
1,Find the average temperature in San Francisco,"As an AI language model, I don't have access t...",False,"[0.39501953125, -0.060089111328125, 3.328125, ..."
2,What is so special about Sir Christopher Wren'...,Sir Christopher Wren's Temple Bar is a histori...,True,"[-0.7021484375, -2.486328125, 3.701171875, -1...."
3,"Nicknamed the 'Pilgrims', this football team b...",The team is the New England Patriots.,False,"[-0.20166015625, -1.0068359375, 2.283203125, -..."
4,"Named for a spring, this city in California ha...","The city you are referring to is called ""Los A...",False,"[0.0848388671875, -0.68212890625, -2.259765625..."
...,...,...,...,...
1985,how many chromosomes are there in a pork chop?,"A pork chop is a cut of meat from a pig, and a...",False,"[1.7763671875, -2.6171875, 3.37109375, -2.2714..."
1986,Describe a fashion trend that is currently pop...,"As an AI language model, I do not have a city ...",False,"[0.76904296875, 0.061248779296875, 5.078125, -..."
1987,Create a speech introducing a friend of yours....,"Ladies and gentlemen, it's my honor to introdu...",True,"[-1.1435546875, -1.529296875, 3.5703125, -0.86..."
1988,"Given a current popular movie, generate a funn...",1. Why did the Black Panther refuse to let the...,False,"[-0.7607421875, 1.0595703125, -3.54296875, -2...."


In [ ]:
# Eliminates any instance without embedding

hallu_factbench_1 = hallu_factbench_1[~hallu_factbench_1['embeddings'].isna()]
hallu_factbench_4 = hallu_factbench_4[~hallu_factbench_4['embeddings'].isna()]
hallu_factbench_8 = hallu_factbench_8[~hallu_factbench_8['embeddings'].isna()]
hallu_factbench_16 = hallu_factbench_16[~hallu_factbench_16['embeddings'].isna()]

In [ ]:
# Evaluates the embedding as list

hallu_factalign_1["embeddings"] = hallu_factalign_1["embeddings"].apply(ast.literal_eval)
hallu_factalign_4["embeddings"] = hallu_factalign_4["embeddings"].apply(ast.literal_eval)
hallu_factalign_8["embeddings"] = hallu_factalign_8["embeddings"].apply(ast.literal_eval)
hallu_factalign_16["embeddings"] = hallu_factalign_16["embeddings"].apply(ast.literal_eval)

hallu_factbench_1["embeddings"] = hallu_factbench_1["embeddings"].apply(ast.literal_eval)
hallu_factbench_4["embeddings"] = hallu_factbench_4["embeddings"].apply(ast.literal_eval)
hallu_factbench_8["embeddings"] = hallu_factbench_8["embeddings"].apply(ast.literal_eval)
hallu_factbench_16["embeddings"] = hallu_factbench_16["embeddings"].apply(ast.literal_eval)

hallu_felm_1["embeddings"] = hallu_felm_1["embeddings"].apply(ast.literal_eval)
hallu_felm_4["embeddings"] = hallu_felm_4["embeddings"].apply(ast.literal_eval)
hallu_felm_8["embeddings"] = hallu_felm_8["embeddings"].apply(ast.literal_eval)
hallu_felm_16["embeddings"] = hallu_felm_16["embeddings"].apply(ast.literal_eval)

In [ ]:
hallu_factalign_1

,prompt,completion,label,embeddings
0,What is the Phaedrus Dialogue?,The Phaedrus Dialogue is a philosophical text ...,False,"[1.171875, -3.705078125, 2.576171875, -2.21679..."
1,What is the Strait of Gibraltar?,The Strait of Gibraltar is a narrow passage be...,True,"[2.095703125, -2.06640625, 2.986328125, -2.580..."
2,What is the Pepsi Challenge?,The Pepsi Challenge is a marketing experiment ...,False,"[0.0218048095703125, -4.296875, 4.5625, -3.509..."
3,What is the Legal Realism movement?,The Legal Realism movement was a legal philoso...,True,"[1.7724609375, -2.4765625, 2.826171875, -1.760..."
4,What occurred during the Tet Offensive?,The Tet Offensive was a surprise military offe...,True,"[0.90625, -1.1005859375, 4.52734375, -3.90625,..."
...,...,...,...,...
2557,What is the Wall Advisory Opinion by the Inter...,The Wall Advisory Opinion by the International...,False,"[2.177734375, 0.19287109375, 1.044921875, -1.9..."
2558,What is the Blue Brain Project?,The Blue Brain Project is a research initiativ...,False,"[-0.51416015625, -3.291015625, 1.6728515625, -..."
2559,Who is Jean Piaget?,Jean Piaget (1896-1980) was a Swiss psychologi...,True,"[-2.7578125, -1.70703125, 1.388671875, -4.2226..."
2560,What occurred with the Cuban Missile Crisis?,The Cuban Missile Crisis was a period of heigh...,False,"[0.32568359375, -2.35546875, 4.34375, -2.58398..."


In [ ]:
hallu_factbench_1

,prompt,response,response_label,embeddings
0,"Generate a poem with length 10-12 lines, based...",The silence of the night \nIs a peaceful sight...,True,"[-1.7470703125, 0.0831298828125, 4.203125, -2...."
1,Find the average temperature in San Francisco,"As an AI language model, I don't have access t...",False,"[0.39501953125, -0.060089111328125, 3.328125, ..."
2,What is so special about Sir Christopher Wren'...,Sir Christopher Wren's Temple Bar is a histori...,True,"[-0.7021484375, -2.486328125, 3.701171875, -1...."
3,"Nicknamed the 'Pilgrims', this football team b...",The team is the New England Patriots.,False,"[-0.20166015625, -1.0068359375, 2.283203125, -..."
4,"Named for a spring, this city in California ha...","The city you are referring to is called ""Los A...",False,"[0.0848388671875, -0.68212890625, -2.259765625..."
...,...,...,...,...
1985,how many chromosomes are there in a pork chop?,"A pork chop is a cut of meat from a pig, and a...",False,"[1.7763671875, -2.6171875, 3.37109375, -2.2714..."
1986,Describe a fashion trend that is currently pop...,"As an AI language model, I do not have a city ...",False,"[0.76904296875, 0.061248779296875, 5.078125, -..."
1987,Create a speech introducing a friend of yours....,"Ladies and gentlemen, it's my honor to introdu...",True,"[-1.1435546875, -1.529296875, 3.5703125, -0.86..."
1988,"Given a current popular movie, generate a funn...",1. Why did the Black Panther refuse to let the...,False,"[-0.7607421875, 1.0595703125, -3.54296875, -2...."


In [ ]:
hallu_felm_1

,prompt,response,hallucination,embeddings
0,"Nicknamed the 'Pilgrims', this football team b...",The team is the New England Patriots.,False,"[-0.20166015625, -1.0068359375, 2.283203125, -..."
1,Where did fortune cookies originate,"""Fortune cookies originated in Japan in the 19...",False,"[0.2021484375, -0.58984375, -1.5732421875, -0...."
2,Mrs. Tatiana owns a grocery store that sells d...,"After one year, the price of carrots would be ...",False,"[1.662109375, -0.107421875, 3.7265625, -2.4160..."
3,What is the average height of female president...,There have been no female presidents of the Un...,True,"[0.6220703125, -1.4775390625, 1.3896484375, -0..."
4,Determine the modulo 4 remainder of the follow...,"We can pair the terms $1$ and $3$, $2$ and $4$...",False,"[1.5546875, -0.31982421875, -0.004611968994140..."
...,...,...,...,...
557,Simplify: $\frac{18!}{16!}$,We can simplify this expression by canceling o...,True,"[1.3857421875, 2.28515625, 0.26806640625, -3.1..."
558,Which is bigger 6 times 3 plus 2 or 6 plus 3 t...,"To solve this problem, we need to follow the o...",True,"[2.46484375, 0.9072265625, 3.0703125, -1.68554..."
559,If Mark weighs 150 pounds and Susan weighs 20 ...,"Susan weighs 20 pounds less than Mark, so she ...",True,"[0.107177734375, -1.0146484375, 3.96484375, -1..."
560,Who was the CEO of Twitter?,The CEO of Twitter at the time of writing this...,False,"[1.220703125, -1.724609375, 2.87109375, -1.366..."


In [ ]:
# Renames the columns in the same way for consistency

hallu_factalign_1 = hallu_factalign_1.rename(columns={'completion': 'response'})
hallu_factalign_4 = hallu_factalign_4.rename(columns={'completion': 'response'})
hallu_factalign_8 = hallu_factalign_8.rename(columns={'completion': 'response'})
hallu_factalign_16 = hallu_factalign_16.rename(columns={'completion': 'response'})

hallu_factbench_1 = hallu_factbench_1.rename(columns={'response_label': 'label'})
hallu_factbench_4 = hallu_factbench_4.rename(columns={'response_label': 'label'})
hallu_factbench_8 = hallu_factbench_8.rename(columns={'response_label': 'label'})
hallu_factbench_16 = hallu_factbench_16.rename(columns={'response_label': 'label'})

hallu_felm_1 = hallu_felm_1.rename(columns={'hallucination': 'label'})
hallu_felm_4 = hallu_felm_4.rename(columns={'hallucination': 'label'})
hallu_felm_8 = hallu_felm_8.rename(columns={'hallucination': 'label'})
hallu_felm_16 = hallu_felm_16.rename(columns={'hallucination': 'label'})

In [ ]:
# Removes any instance without factuality label

hallu_factbench_1 = hallu_factbench_1[~hallu_factbench_1['label'].isna()]
hallu_factbench_4 = hallu_factbench_4[~hallu_factbench_4['label'].isna()]
hallu_factbench_8 = hallu_factbench_8[~hallu_factbench_8['label'].isna()]
hallu_factbench_16 = hallu_factbench_16[~hallu_factbench_16['label'].isna()]

In [ ]:
# Convert True and False in 1/0

hallu_factbench_1['label'] = hallu_factbench_1['label'].astype(int)
hallu_factbench_4['label'] = hallu_factbench_4['label'].astype(int)
hallu_factbench_8['label'] = hallu_factbench_8['label'].astype(int)
hallu_factbench_16['label'] = hallu_factbench_16['label'].astype(int)

hallu_felm_1['label'] = hallu_felm_1['label'].astype(int)
hallu_felm_4['label'] = hallu_felm_4['label'].astype(int)
hallu_felm_8['label'] = hallu_felm_8['label'].astype(int)
hallu_felm_16['label'] = hallu_felm_16['label'].astype(int)

hallu_factalign_1['label'] = hallu_factalign_1['label'].astype(int)
hallu_factalign_4['label'] = hallu_factalign_4['label'].astype(int)
hallu_factalign_8['label'] = hallu_factalign_8['label'].astype(int)
hallu_factalign_16['label'] = hallu_factalign_16['label'].astype(int)

## **Training**

In [ ]:
# Split the data in train and test, using a random seed for consistency

random.seed(777)

felm_train_1, felm_test_1 = train_test_split(hallu_felm_1, test_size=0.2, random_state=42)
felm_train_4, felm_test_4 = train_test_split(hallu_felm_4, test_size=0.2, random_state=42)
felm_train_8, felm_test_8 = train_test_split(hallu_felm_8, test_size=0.2, random_state=42)
felm_train_16, felm_test_16 = train_test_split(hallu_felm_16, test_size=0.2, random_state=42)

factalign_train_1, factalign_test_1 = train_test_split(hallu_factalign_1, test_size=0.2, random_state=42)
factalign_train_4, factalign_test_4 = train_test_split(hallu_factalign_4, test_size=0.2, random_state=42)
factalign_train_8, factalign_test_8 = train_test_split(hallu_factalign_8, test_size=0.2, random_state=42)
factalign_train_16, factalign_test_16 = train_test_split(hallu_factalign_16, test_size=0.2, random_state=42)

factbench_train_1, factbench_test_1 = train_test_split(hallu_factbench_1, test_size=0.2, random_state=42)
factbench_train_4, factbench_test_4 = train_test_split(hallu_factbench_4, test_size=0.2, random_state=42)
factbench_train_8, factbench_test_8 = train_test_split(hallu_factbench_8, test_size=0.2, random_state=42)
factbench_train_16, factbench_test_16 = train_test_split(hallu_factbench_16, test_size=0.2, random_state=42)

In [ ]:
# Transform the embedding in numpy array

## FELM

X_train_felm_1 = np.array(felm_train_1['embeddings'].tolist())
X_test_felm_1 = np.array(felm_test_1['embeddings'].tolist())

X_train_felm_4 = np.array(felm_train_4['embeddings'].tolist())
X_test_felm_4 = np.array(felm_test_4['embeddings'].tolist())

X_train_felm_8 = np.array(felm_train_8['embeddings'].tolist())
X_test_felm_8 = np.array(felm_test_8['embeddings'].tolist())

X_train_felm_16 = np.array(felm_train_16['embeddings'].tolist())
X_test_felm_16 = np.array(felm_test_16['embeddings'].tolist())

X_train_felm_all = np.concatenate((X_train_felm_1, X_train_felm_4, X_train_felm_8, X_train_felm_16), axis=1)
X_test_felm_all = np.concatenate((X_test_felm_1, X_test_felm_4, X_test_felm_8, X_test_felm_16), axis=1)

## FACTALIGN

X_train_factalign_1 = np.array(factalign_train_1['embeddings'].tolist())
X_test_factalign_1 = np.array(factalign_test_1['embeddings'].tolist())

X_train_factalign_4 = np.array(factalign_train_4['embeddings'].tolist())
X_test_factalign_4 = np.array(factalign_test_4['embeddings'].tolist())

X_train_factalign_8 = np.array(factalign_train_8['embeddings'].tolist())
X_test_factalign_8 = np.array(factalign_test_8['embeddings'].tolist())

X_train_factalign_16 = np.array(factalign_train_16['embeddings'].tolist())
X_test_factalign_16 = np.array(factalign_test_16['embeddings'].tolist())

X_train_factalign_all = np.concatenate((X_train_factalign_1, X_train_factalign_4, X_train_factalign_8, X_train_factalign_16), axis=1)
X_test_factalign_all = np.concatenate((X_test_factalign_1, X_test_factalign_4, X_test_factalign_8, X_test_factalign_16), axis=1)

## FACTBENCH

X_train_factbench_1 = np.array(factbench_train_1['embeddings'].tolist())
X_test_factbench_1 = np.array(factbench_test_1['embeddings'].tolist())

X_train_factbench_4 = np.array(factbench_train_4['embeddings'].tolist())
X_test_factbench_4 = np.array(factbench_test_4['embeddings'].tolist())

X_train_factbench_8 = np.array(factbench_train_8['embeddings'].tolist())
X_test_factbench_8 = np.array(factbench_test_8['embeddings'].tolist())

X_train_factbench_16 = np.array(factbench_train_16['embeddings'].tolist())
X_test_factbench_16 = np.array(factbench_test_16['embeddings'].tolist())

X_train_factbench_all = np.concatenate((X_train_factbench_1, X_train_factbench_4, X_train_factbench_8, X_train_factbench_16), axis=1)
X_test_factbench_all = np.concatenate((X_test_factbench_1, X_test_factbench_4, X_test_factbench_8, X_test_factbench_16), axis=1)

In [ ]:
# Transform the list of labels in numpy array

## FELM

y_train_felm_1 = np.array(felm_train_1['label'].tolist())
y_test_felm_1 = np.array(felm_test_1['label'].tolist())

y_train_felm_4 = np.array(felm_train_4['label'].tolist())
y_test_felm_4 = np.array(felm_test_4['label'].tolist())

y_train_felm_8 = np.array(felm_train_8['label'].tolist())
y_test_felm_8 = np.array(felm_test_8['label'].tolist())

y_train_felm_16 = np.array(felm_train_16['label'].tolist())
y_test_felm_16 = np.array(felm_test_16['label'].tolist())

## FACTALIGN

y_train_factalign_1 = np.array(factalign_train_1['label'].tolist())
y_test_factalign_1 = np.array(factalign_test_1['label'].tolist())

y_train_factalign_4 = np.array(factalign_train_4['label'].tolist())
y_test_factalign_4 = np.array(factalign_test_4['label'].tolist())

y_train_factalign_8 = np.array(factalign_train_8['label'].tolist())
y_test_factalign_8 = np.array(factalign_test_8['label'].tolist())

y_train_factalign_16 = np.array(factalign_train_16['label'].tolist())
y_test_factalign_16 = np.array(factalign_test_16['label'].tolist())

## FACTBENCH

y_train_factbench_1 = np.array(factbench_train_1['label'].tolist())
y_test_factbench_1 = np.array(factbench_test_1['label'].tolist())

y_train_factbench_4 = np.array(factbench_train_4['label'].tolist())
y_test_factbench_4 = np.array(factbench_test_4['label'].tolist())

y_train_factbench_8 = np.array(factbench_train_8['label'].tolist())
y_test_factbench_8 = np.array(factbench_test_8['label'].tolist())

y_train_factbench_16 = np.array(factbench_train_16['label'].tolist())
y_test_factbench_16 = np.array(factbench_test_16['label'].tolist())

## **SVM**

In [ ]:
# Defining all the SVM classifiers for the classification of the hidden states

clf_felm_1 = SVC()
clf_factalign_1 = SVC()
clf_factbench_1 = SVC()

clf_felm_4 = SVC()
clf_factalign_4 = SVC()
clf_factbench_4 = SVC()

clf_felm_8 = SVC()
clf_factalign_8 = SVC()
clf_factbench_8 = SVC()

clf_felm_16 = SVC()
clf_factalign_16 = SVC()
clf_factbench_16 = SVC()

clf_felm_all = SVC()
clf_factalign_all = SVC()
clf_factbench_all = SVC()

In [ ]:
# Training of the SVM classifiers

clf_felm_1.fit(X_train_felm_1, y_train_felm_1)
clf_factalign_1.fit(X_train_factalign_1, y_train_factalign_1)
clf_factbench_1.fit(X_train_factbench_1, y_train_factbench_1)

clf_felm_4.fit(X_train_felm_4, y_train_felm_4)
clf_factalign_4.fit(X_train_factalign_4, y_train_factalign_4)
clf_factbench_4.fit(X_train_factbench_4, y_train_factbench_4)

clf_felm_8.fit(X_train_felm_8, y_train_felm_8)
clf_factalign_8.fit(X_train_factalign_8, y_train_factalign_8)
clf_factbench_8.fit(X_train_factbench_8, y_train_factbench_8)

clf_felm_16.fit(X_train_felm_16, y_train_felm_16)
clf_factalign_16.fit(X_train_factalign_16, y_train_factalign_16)
clf_factbench_16.fit(X_train_factbench_16, y_train_factbench_16)

clf_felm_all.fit(X_train_felm_all, y_train_felm_1)
clf_factalign_all.fit(X_train_factalign_all, y_train_factalign_1)
clf_factbench_all.fit(X_train_factbench_all, y_train_factbench_1)

SVC()

In [ ]:
# Computes the results using the test set against the ground truths

result_svm_felm_1 = accuracy_score(y_test_felm_1, clf_felm_1.predict(X_test_felm_1))
result_svm_felm_4 = accuracy_score(y_test_felm_4, clf_felm_4.predict(X_test_felm_4))
result_svm_felm_8 = accuracy_score(y_test_felm_8, clf_felm_8.predict(X_test_felm_8))
result_svm_felm_16 = accuracy_score(y_test_felm_16, clf_felm_16.predict(X_test_felm_16))

result_svm_factalign_1 = accuracy_score(y_test_factalign_1, clf_factalign_1.predict(X_test_factalign_1))
result_svm_factalign_4 = accuracy_score(y_test_factalign_4, clf_factalign_4.predict(X_test_factalign_4))
result_svm_factalign_8 = accuracy_score(y_test_factalign_8, clf_factalign_8.predict(X_test_factalign_8))
result_svm_factalign_16 = accuracy_score(y_test_factalign_16, clf_factalign_16.predict(X_test_factalign_16))

result_svm_factbench_1 = accuracy_score(y_test_factbench_1, clf_factbench_1.predict(X_test_factbench_1))
result_svm_factbench_4 = accuracy_score(y_test_factbench_4, clf_factbench_4.predict(X_test_factbench_4))
result_svm_factbench_8 = accuracy_score(y_test_factbench_8, clf_factbench_8.predict(X_test_factbench_8))
result_svm_factbench_16 = accuracy_score(y_test_factbench_16, clf_factbench_16.predict(X_test_factbench_16))

result_svm_felm_all = accuracy_score(y_test_felm_1, clf_felm_all.predict(X_test_felm_all))
result_svm_factalign_all = accuracy_score(y_test_factalign_1, clf_factalign_all.predict(X_test_factalign_all))
result_svm_factbench_all = accuracy_score(y_test_factbench_1, clf_factbench_all.predict(X_test_factbench_all))

## **Logistic Regression**

In [ ]:
# Defining all the Logistic Regression classifiers for the classification of the hidden states

lr_felm_1 = LogisticRegression()
lr_factalign_1 = LogisticRegression()
lr_factbench_1 = LogisticRegression()

lr_felm_4 = LogisticRegression()
lr_factalign_4 = LogisticRegression()
lr_factbench_4 = LogisticRegression()

lr_felm_8 = LogisticRegression()
lr_factalign_8 = LogisticRegression()
lr_factbench_8 = LogisticRegression()

lr_felm_16 = LogisticRegression()
lr_factalign_16 = LogisticRegression()
lr_factbench_16 = LogisticRegression()

lr_felm_all = LogisticRegression()
lr_factalign_all = LogisticRegression()
lr_factbench_all = LogisticRegression()

In [ ]:
# Training of the logistic regression classifiers

lr_felm_1.fit(X_train_felm_1, y_train_felm_1)
lr_factalign_1.fit(X_train_factalign_1, y_train_factalign_1)
lr_factbench_1.fit(X_train_factbench_1, y_train_factbench_1)

lr_felm_4.fit(X_train_felm_4, y_train_felm_4)
lr_factalign_4.fit(X_train_factalign_4, y_train_factalign_4)
lr_factbench_4.fit(X_train_factbench_4, y_train_factbench_4)

lr_felm_8.fit(X_train_felm_8, y_train_felm_8)
lr_factalign_8.fit(X_train_factalign_8, y_train_factalign_8)
lr_factbench_8.fit(X_train_factbench_8, y_train_factbench_8)

lr_felm_16.fit(X_train_felm_16, y_train_felm_16)
lr_factalign_16.fit(X_train_factalign_16, y_train_factalign_16)
lr_factbench_16.fit(X_train_factbench_16, y_train_factbench_16)

lr_felm_all.fit(X_train_felm_all, y_train_felm_1)
lr_factalign_all.fit(X_train_factalign_all, y_train_factalign_1)
lr_factbench_all.fit(X_train_factbench_all, y_train_factbench_1)

LogisticRegression()

In [ ]:
# Computes the results using the test set against the ground truths

result_lr_felm_1 = accuracy_score(y_test_felm_1, lr_felm_1.predict(X_test_felm_1))
result_lr_felm_4 = accuracy_score(y_test_felm_4, lr_felm_4.predict(X_test_felm_4))
result_lr_felm_8 = accuracy_score(y_test_felm_8, lr_felm_8.predict(X_test_felm_8))
result_lr_felm_16 = accuracy_score(y_test_felm_16, lr_felm_16.predict(X_test_felm_16))

result_lr_factalign_1 = accuracy_score(y_test_factalign_1, lr_factalign_1.predict(X_test_factalign_1))
result_lr_factalign_4 = accuracy_score(y_test_factalign_4, lr_factalign_4.predict(X_test_factalign_4))
result_lr_factalign_8 = accuracy_score(y_test_factalign_8, lr_factalign_8.predict(X_test_factalign_8))
result_lr_factalign_16 = accuracy_score(y_test_factalign_16, lr_factalign_16.predict(X_test_factalign_16))

result_lr_factbench_1 = accuracy_score(y_test_factbench_1, lr_factbench_1.predict(X_test_factbench_1))
result_lr_factbench_4 = accuracy_score(y_test_factbench_4, lr_factbench_4.predict(X_test_factbench_4))
result_lr_factbench_8 = accuracy_score(y_test_factbench_8, lr_factbench_8.predict(X_test_factbench_8))
result_lr_factbench_16 = accuracy_score(y_test_factbench_16, lr_factbench_16.predict(X_test_factbench_16))

result_lr_felm_all = accuracy_score(y_test_felm_1, lr_felm_all.predict(X_test_felm_all))
result_lr_factalign_all = accuracy_score(y_test_factalign_1, lr_factalign_all.predict(X_test_factalign_all))
result_lr_factbench_all = accuracy_score(y_test_factbench_1, lr_factbench_all.predict(X_test_factbench_all))

## **Dataframes with metrics**

In [ ]:
metrics_felm_svm = {
    'classifier' : ['svm', 'svm', 'svm', 'svm', 'svm'],
    'layer' : [-1, -4, -8, -16, 'all'],
    'accuracy' : [result_svm_felm_1, result_svm_felm_4, result_svm_felm_8, result_svm_felm_16, result_svm_felm_all]
}

metrics_felm_lr = {
    'classifier' : ['lr', 'lr', 'lr', 'lr', 'lr'],
    'layer' : [-1, -4, -8, -16, 'all'],
    'accuracy' : [result_lr_felm_1, result_lr_felm_4, result_lr_felm_8, result_lr_felm_16, result_lr_felm_all]
}

metrics_factalign_svm = {
    'classifier' : ['svm', 'svm', 'svm', 'svm', 'svm'],
    'layer' : [-1, -4, -8, -16, 'all'],
    'accuracy' : [result_svm_factalign_1, result_svm_factalign_4, result_svm_factalign_8, result_svm_factalign_16, result_svm_factalign_all]
}

metrics_factalign_lr = {
    'classifier' : ['lr', 'lr', 'lr', 'lr', 'lr'],
    'layer' : [-1, -4, -8, -16, 'all'],
    'accuracy' : [result_lr_factalign_1, result_lr_factalign_4, result_lr_factalign_8, result_lr_factalign_16, result_lr_factalign_all]
}

metrics_factbench_svm = {
    'classifier' : ['svm', 'svm', 'svm', 'svm', 'svm'],
    'layer' : [-1, -4, -8, -16, 'all'],
    'accuracy' : [result_svm_factbench_1, result_svm_factbench_4, result_svm_factbench_8, result_svm_factbench_16, result_svm_factbench_all]
}

metrics_factbench_lr = {
    'classifier' : ['lr', 'lr', 'lr', 'lr', 'lr'],
    'layer' : [-1, -4, -8, -16, 'all'],
    'accuracy' : [result_lr_factbench_1, result_lr_factbench_4, result_lr_factbench_8, result_lr_factbench_16, result_lr_factbench_all]
}

In [ ]:
df_metrics_felm_svm = pd.DataFrame(metrics_felm_svm)
df_metrics_felm_lr = pd.DataFrame(metrics_felm_lr)

df_metrics_factalign_svm = pd.DataFrame(metrics_factalign_svm)
df_metrics_factalign_lr = pd.DataFrame(metrics_factalign_lr)

df_metrics_factbench_svm = pd.DataFrame(metrics_factbench_svm)
df_metrics_factbench_lr = pd.DataFrame(metrics_factbench_lr)

In [ ]:
df_metrics_felm = pd.concat([df_metrics_felm_svm, df_metrics_felm_lr])
df_metrics_factalign = pd.concat([df_metrics_factalign_svm, df_metrics_factalign_lr])
df_metrics_factbench = pd.concat([df_metrics_factbench_svm, df_metrics_factbench_lr])

In [ ]:
hallu_felm_1

,prompt,response,label,embeddings
0,"Nicknamed the 'Pilgrims', this football team b...",The team is the New England Patriots.,0,"[-0.20166015625, -1.0068359375, 2.283203125, -..."
1,Where did fortune cookies originate,"""Fortune cookies originated in Japan in the 19...",0,"[0.2021484375, -0.58984375, -1.5732421875, -0...."
2,Mrs. Tatiana owns a grocery store that sells d...,"After one year, the price of carrots would be ...",0,"[1.662109375, -0.107421875, 3.7265625, -2.4160..."
3,What is the average height of female president...,There have been no female presidents of the Un...,1,"[0.6220703125, -1.4775390625, 1.3896484375, -0..."
4,Determine the modulo 4 remainder of the follow...,"We can pair the terms $1$ and $3$, $2$ and $4$...",0,"[1.5546875, -0.31982421875, -0.004611968994140..."
...,...,...,...,...
557,Simplify: $\frac{18!}{16!}$,We can simplify this expression by canceling o...,1,"[1.3857421875, 2.28515625, 0.26806640625, -3.1..."
558,Which is bigger 6 times 3 plus 2 or 6 plus 3 t...,"To solve this problem, we need to follow the o...",1,"[2.46484375, 0.9072265625, 3.0703125, -1.68554..."
559,If Mark weighs 150 pounds and Susan weighs 20 ...,"Susan weighs 20 pounds less than Mark, so she ...",1,"[0.107177734375, -1.0146484375, 3.96484375, -1..."
560,Who was the CEO of Twitter?,The CEO of Twitter at the time of writing this...,0,"[1.220703125, -1.724609375, 2.87109375, -1.366..."


## **Neural Network - 3 layers (SAPLMA)**

In [ ]:
## Function that define the deep neural networks with three layers

def define_model_3layer(input_dim = X_train_felm_1.shape[1]):

  model = Sequential()
  model.add(Dense(256, activation='relu', input_dim=input_dim))
  model.add(Dense(128, activation='relu'))
  model.add(Dense(64, activation='relu'))
  model.add(Dense(1, activation='relu'))
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

  return model

#### **Training of the 3-layers Neural Networks**

In [ ]:
model = define_model_3layer()
model.fit(X_train_felm_1, y_train_felm_1, epochs=5, batch_size=32)
loss_felm_1, accuracy_felm_1 = model.evaluate(X_test_felm_1, y_test_felm_1)

Epoch 1/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 3s 45ms/step - accuracy: 0.4659 - loss: 8.0373
Epoch 2/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4997 - loss: 8.0644  
Epoch 3/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4874 - loss: 8.2629 
Epoch 4/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5131 - loss: 7.8481 
Epoch 5/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4962 - loss: 8.1202 
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 87ms/step - accuracy: 0.4668 - loss: 8.5945


In [ ]:
model = define_model_3layer()
model.fit(X_train_felm_4, y_train_felm_4, epochs=5, batch_size=32)
loss_felm_4, accuracy_felm_4 = model.evaluate(X_test_felm_4, y_test_felm_4)

Epoch 1/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 3s 35ms/step - accuracy: 0.5278 - loss: 5.7128
Epoch 2/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4835 - loss: 8.3246 
Epoch 3/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5037 - loss: 7.9994 
Epoch 4/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5174 - loss: 7.7780 
Epoch 5/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5094 - loss: 7.9080 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.4668 - loss: 8.5945


In [ ]:
model = define_model_3layer()
model.fit(X_train_felm_8, y_train_felm_8, epochs=5, batch_size=32)
loss_felm_8, accuracy_felm_8 = model.evaluate(X_test_felm_8, y_test_felm_8)

Epoch 1/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 36ms/step - accuracy: 0.4520 - loss: 4.9091
Epoch 2/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5603 - loss: 5.0247 
Epoch 3/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5980 - loss: 1.2776 
Epoch 4/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5792 - loss: 0.8550 
Epoch 5/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6860 - loss: 0.6184 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.6118 - loss: 0.6643


In [ ]:
model = define_model_3layer()
model.fit(X_train_felm_16, y_train_felm_16, epochs=5, batch_size=32)
loss_felm_16, accuracy_felm_16 = model.evaluate(X_test_felm_16, y_test_felm_16)

Epoch 1/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 36ms/step - accuracy: 0.5514 - loss: 0.9427
Epoch 2/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6023 - loss: 0.6701 
Epoch 3/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7037 - loss: 0.5045 
Epoch 4/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8439 - loss: 0.3546 
Epoch 5/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9148 - loss: 0.2092 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.6357 - loss: 1.3583


In [ ]:
model = define_model_3layer(X_train_felm_all.shape[1])
model.fit(X_train_felm_all, y_train_felm_1, epochs=5, batch_size=32)
loss_felm_all, accuracy_felm_all = model.evaluate(X_test_felm_all, y_test_felm_1)

Epoch 1/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 3s 45ms/step - accuracy: 0.5273 - loss: 7.3188
Epoch 2/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4578 - loss: 8.6443  
Epoch 3/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4844 - loss: 8.2201 
Epoch 4/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5004 - loss: 7.9654 
Epoch 5/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5002 - loss: 7.9681 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.5332 - loss: 7.4415


In [ ]:
model = define_model_3layer()
model.fit(X_train_factalign_1, y_train_factalign_1, epochs=5, batch_size=32)
loss_factalign_1, accuracy_factalign_1 = model.evaluate(X_test_factalign_1, y_test_factalign_1)

Epoch 1/5
65/65 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.4815 - loss: 8.1667
Epoch 2/5
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4883 - loss: 8.2477
Epoch 3/5
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4861 - loss: 8.2824
Epoch 4/5
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4728 - loss: 8.4976
Epoch 5/5
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4764 - loss: 8.4401
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.5171 - loss: 7.7831


In [ ]:
model = define_model_3layer()
model.fit(X_train_factalign_4, y_train_factalign_4, epochs=5, batch_size=32)
loss_factalign_4, accuracy_factalign_4 = model.evaluate(X_test_factalign_4, y_test_factalign_4)

Epoch 1/5
65/65 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.5181 - loss: 6.8817
Epoch 2/5
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5069 - loss: 7.8612
Epoch 3/5
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5073 - loss: 7.8540
Epoch 4/5
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5255 - loss: 7.5644
Epoch 5/5
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5127 - loss: 7.7691
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.4829 - loss: 8.2441


In [ ]:
model = define_model_3layer()
model.fit(X_train_factalign_8, y_train_factalign_8, epochs=5, batch_size=32)
loss_factalign_8, accuracy_factalign_8 = model.evaluate(X_test_factalign_8, y_test_factalign_8)

Epoch 1/5
65/65 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.5184 - loss: 2.4317
Epoch 2/5
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5190 - loss: 5.4032
Epoch 3/5
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5257 - loss: 7.4570
Epoch 4/5
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5260 - loss: 7.5292
Epoch 5/5
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5437 - loss: 7.2551
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.4851 - loss: 8.1425


In [ ]:
model = define_model_3layer()
model.fit(X_train_factalign_16, y_train_factalign_16, epochs=5, batch_size=32)
loss_factalign_16, accuracy_factalign_16 = model.evaluate(X_test_factalign_16, y_test_factalign_16)

Epoch 1/5
65/65 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.5457 - loss: 0.9800
Epoch 2/5
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6878 - loss: 0.6000
Epoch 3/5
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6255 - loss: 0.6400
Epoch 4/5
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7861 - loss: 0.5342
Epoch 5/5
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7546 - loss: 0.6511
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.6487 - loss: 0.7707


In [ ]:
model = define_model_3layer(X_train_factalign_all.shape[1])
model.fit(X_train_factalign_all, y_train_factalign_1, epochs=5, batch_size=32)
loss_factalign_all, accuracy_factalign_all = model.evaluate(X_test_factalign_all, y_test_factalign_1)

Epoch 1/5
65/65 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.5231 - loss: 7.4828
Epoch 2/5
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5232 - loss: 7.6014
Epoch 3/5
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5260 - loss: 7.5561
Epoch 4/5
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5038 - loss: 7.9102
Epoch 5/5
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5011 - loss: 7.9530
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.4829 - loss: 8.2441


In [ ]:
model = define_model_3layer()
model.fit(X_train_factbench_1, y_train_factbench_1, epochs=5, batch_size=32)
loss_factbench_1, accuracy_factbench_1 = model.evaluate(X_test_factbench_1, y_test_factbench_1)

Epoch 1/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - accuracy: 0.4997 - loss: 7.6606
Epoch 2/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5150 - loss: 7.8179
Epoch 3/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5213 - loss: 7.7157
Epoch 4/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5049 - loss: 7.9805
Epoch 5/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5133 - loss: 7.8449
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.5119 - loss: 7.8674


In [ ]:
model = define_model_3layer()
model.fit(X_train_factbench_4, y_train_factbench_4, epochs=5, batch_size=32)
loss_factbench_4, accuracy_factbench_4 = model.evaluate(X_test_factbench_4, y_test_factbench_4)

Epoch 1/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.5249 - loss: 7.3739
Epoch 2/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5025 - loss: 8.0192
Epoch 3/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5058 - loss: 7.9658
Epoch 4/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4842 - loss: 8.3137
Epoch 5/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5033 - loss: 8.0056
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.5119 - loss: 7.8674


In [ ]:
model = define_model_3layer()
model.fit(X_train_factbench_8, y_train_factbench_8, epochs=5, batch_size=32)
loss_factbench_8, accuracy_factbench_8 = model.evaluate(X_test_factbench_8, y_test_factbench_8)

Epoch 1/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.4855 - loss: 7.3747
Epoch 2/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4921 - loss: 8.0938
Epoch 3/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4886 - loss: 7.9568
Epoch 4/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5154 - loss: 7.8107
Epoch 5/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4940 - loss: 8.1557
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.5119 - loss: 7.8674


In [ ]:
model = define_model_3layer()
model.fit(X_train_factbench_16, y_train_factbench_16, epochs=5, batch_size=32)
loss_factbench_16, accuracy_factbench_16 = model.evaluate(X_test_factbench_16, y_test_factbench_16)

Epoch 1/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.5496 - loss: 1.0276
Epoch 2/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7435 - loss: 0.6308
Epoch 3/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7008 - loss: 0.7209
Epoch 4/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8008 - loss: 0.5750
Epoch 5/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8087 - loss: 0.5010
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.6028 - loss: 1.5886


In [ ]:
model = define_model_3layer(X_train_factbench_all.shape[1])
model.fit(X_train_factbench_all, y_train_factbench_1, epochs=5, batch_size=32)
loss_factbench_all, accuracy_factbench_all = model.evaluate(X_test_factbench_all, y_test_factbench_1)

Epoch 1/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.5038 - loss: 7.7151
Epoch 2/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4952 - loss: 8.0473
Epoch 3/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4912 - loss: 8.1122
Epoch 4/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4848 - loss: 8.2140
Epoch 5/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4874 - loss: 8.1724
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.4881 - loss: 8.1608


## **Dataframes with metrics**

In [ ]:
metrics_felm_3layer = {
    'classifier' : ['dnn_3_layer', 'dnn_3_layer', 'dnn_3_layer', 'dnn_3_layer', 'dnn_3_layer'],
    'layer' : [-1, -4, -8, -16, 'all'],
    'accuracy' : [accuracy_felm_1, accuracy_felm_4, accuracy_felm_8, accuracy_felm_16, accuracy_felm_all]
}

metrics_factalign_3layer = {
    'classifier' : ['dnn_3_layer', 'dnn_3_layer', 'dnn_3_layer', 'dnn_3_layer', 'dnn_3_layer'],
    'layer' : [-1, -4, -8, -16, 'all'],
    'accuracy' : [accuracy_factalign_1, accuracy_factalign_4, accuracy_factalign_8, accuracy_factalign_16, accuracy_factalign_all]
}

metrics_factbench_3layer = {
    'classifier' : ['dnn_3_layer', 'dnn_3_layer', 'dnn_3_layer', 'dnn_3_layer', 'dnn_3_layer'],
    'layer' : [-1, -4, -8, -16, 'all'],
    'accuracy' : [accuracy_factbench_1, accuracy_factbench_4, accuracy_factbench_8, accuracy_factbench_16, accuracy_factbench_all]
}


In [ ]:
df_metrics_felm_3layer = pd.DataFrame(metrics_felm_3layer)
df_metrics_factalign_3layer = pd.DataFrame(metrics_factalign_3layer)
df_metrics_factbench_3layer = pd.DataFrame(metrics_factbench_3layer)

In [ ]:
df_metrics_felm = pd.concat([df_metrics_felm, df_metrics_felm_3layer])
df_metrics_factalign = pd.concat([df_metrics_factalign, df_metrics_factalign_3layer])
df_metrics_factbench = pd.concat([df_metrics_factbench, df_metrics_factbench_3layer])

In [ ]:
df_metrics_factalign

,classifier,layer,accuracy
0,svm,-1,0.730994
1,svm,-4,0.748538
2,svm,-8,0.717349
3,svm,-16,0.719298
4,svm,all,0.736842
0,lr,-1,0.693957
1,lr,-4,0.703704
2,lr,-8,0.701754
3,lr,-16,0.727096
4,lr,all,0.725146


## **Neural Network - 5 layers**

In [ ]:
## Function that define the deep neural networks with five layers

def define_model_5layer(input_dim = X_train_felm_1.shape[1]):

  model = Sequential()
  model.add(Dense(1024, activation='relu', input_dim=input_dim))
  model.add(Dense(512, activation='relu', input_dim=input_dim))
  model.add(Dense(256, activation='relu', input_dim=input_dim))
  model.add(Dense(128, activation='relu'))
  model.add(Dense(64, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

  return model

#### **Training of the 5-layers Neural Networks**

In [ ]:
model = define_model_5layer()
model.fit(X_train_felm_1, y_train_felm_1, epochs=5, batch_size=32)
loss_felm_1_5layer, accuracy_felm_1_5layer = model.evaluate(X_test_felm_1, y_test_felm_1)

model = define_model_5layer()
model.fit(X_train_felm_4, y_train_felm_4, epochs=5, batch_size=32)
loss_felm_4_5layer, accuracy_felm_4_5layer = model.evaluate(X_test_felm_4, y_test_felm_4)

model = define_model_5layer()
model.fit(X_train_felm_8, y_train_felm_8, epochs=5, batch_size=32)
loss_felm_8_5layer, accuracy_felm_8_5layer = model.evaluate(X_test_felm_8, y_test_felm_8)

model = define_model_5layer()
model.fit(X_train_felm_16, y_train_felm_16, epochs=5, batch_size=32)
loss_felm_16_5layer, accuracy_felm_16_5layer = model.evaluate(X_test_felm_16, y_test_felm_16)

model = define_model_5layer(X_train_felm_all.shape[1])
model.fit(X_train_felm_all, y_train_felm_1, epochs=5, batch_size=32)
loss_felm_all_5layer, accuracy_felm_all_5layer = model.evaluate(X_test_felm_all, y_test_felm_1)

model = define_model_5layer()
model.fit(X_train_factalign_1, y_train_factalign_1, epochs=5, batch_size=32)
loss_factalign_1_5layer, accuracy_factalign_1_5layer = model.evaluate(X_test_factalign_1, y_test_factalign_1)

model = define_model_5layer()
model.fit(X_train_factalign_4, y_train_factalign_4, epochs=5, batch_size=32)
loss_factalign_4_5layer, accuracy_factalign_4_5layer = model.evaluate(X_test_factalign_4, y_test_factalign_4)

model = define_model_5layer()
model.fit(X_train_factalign_8, y_train_factalign_8, epochs=5, batch_size=32)
loss_factalign_8_5layer, accuracy_factalign_8_5layer = model.evaluate(X_test_factalign_8, y_test_factalign_8)

model = define_model_5layer()
model.fit(X_train_factalign_16, y_train_factalign_16, epochs=5, batch_size=32)
loss_factalign_16_5layer, accuracy_factalign_16_5layer = model.evaluate(X_test_factalign_16, y_test_factalign_16)

model = define_model_5layer(X_train_factalign_all.shape[1])
model.fit(X_train_factalign_all, y_train_factalign_1, epochs=5, batch_size=32)
loss_factalign_all_5layer, accuracy_factalign_all_5layer = model.evaluate(X_test_factalign_all, y_test_factalign_1)

model = define_model_5layer()
model.fit(X_train_factbench_1, y_train_factbench_1, epochs=5, batch_size=32)
loss_factbench_1_5layer, accuracy_factbench_1_5layer = model.evaluate(X_test_factbench_1, y_test_factbench_1)

model = define_model_5layer()
model.fit(X_train_factbench_4, y_train_factbench_4, epochs=5, batch_size=32)
loss_factbench_4_5layer, accuracy_factbench_4_5layer = model.evaluate(X_test_factbench_4, y_test_factbench_4)

model = define_model_5layer()
model.fit(X_train_factbench_8, y_train_factbench_8, epochs=5, batch_size=32)
loss_factbench_8_5layer, accuracy_factbench_8_5layer = model.evaluate(X_test_factbench_8, y_test_factbench_8)

model = define_model_5layer()
model.fit(X_train_factbench_16, y_train_factbench_16, epochs=5, batch_size=32)
loss_factbench_16_5layer, accuracy_factbench_16_5layer = model.evaluate(X_test_factbench_16, y_test_factbench_16)

model = define_model_5layer(X_train_factbench_all.shape[1])
model.fit(X_train_factbench_all, y_train_factbench_1, epochs=5, batch_size=32)
loss_factbench_all_5layer, accuracy_factbench_all_5layer = model.evaluate(X_test_factbench_all, y_test_factbench_1)

Epoch 1/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 4s 71ms/step - accuracy: 0.5153 - loss: 1.0106
Epoch 2/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5757 - loss: 0.7523 
Epoch 3/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6868 - loss: 0.6323 
Epoch 4/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5948 - loss: 0.7686 
Epoch 5/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6116 - loss: 0.6779 
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 123ms/step - accuracy: 0.5053 - loss: 0.7768
Epoch 1/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 3s 55ms/step - accuracy: 0.5363 - loss: 0.7620
Epoch 2/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6130 - loss: 0.6710 
Epoch 3/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6788 - loss: 0.6346 
Epoch 4/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7707 - loss: 0.4536 
Epoch 5/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7874 - loss: 0.4671 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 0.5834 - loss: 1.2869
Epoch 1/5
15/15

## **Dataframes with metrics**

In [ ]:
metrics_felm_5layer = {
    'classifier' : ['dnn_5_layer', 'dnn_5_layer', 'dnn_5_layer', 'dnn_5_layer', 'dnn_5_layer'],
    'layer' : [-1, -4, -8, -16, 'all'],
    'accuracy' : [accuracy_felm_1_5layer, accuracy_felm_4_5layer, accuracy_felm_8_5layer, accuracy_felm_16_5layer, accuracy_felm_all_5layer]
}

metrics_factalign_5layer = {
    'classifier' : ['dnn_5_layer', 'dnn_5_layer', 'dnn_5_layer', 'dnn_5_layer', 'dnn_5_layer'],
    'layer' : [-1, -4, -8, -16, 'all'],
    'accuracy' : [accuracy_factalign_1_5layer, accuracy_factalign_4_5layer, accuracy_factalign_8_5layer, accuracy_factalign_16_5layer, accuracy_factalign_all_5layer]
}

metrics_factbench_5layer = {
    'classifier' : ['dnn_5_layer', 'dnn_5_layer', 'dnn_5_layer', 'dnn_5_layer', 'dnn_5_layer'],
    'layer' : [-1, -4, -8, -16, 'all'],
    'accuracy' : [accuracy_factbench_1_5layer, accuracy_factbench_4_5layer, accuracy_factbench_8_5layer, accuracy_factbench_16_5layer, accuracy_factbench_all_5layer]
}


In [ ]:
df_metrics_felm_5layer = pd.DataFrame(metrics_felm_5layer)
df_metrics_factalign_5layer = pd.DataFrame(metrics_factalign_5layer)
df_metrics_factbench_5layer = pd.DataFrame(metrics_factbench_5layer)

In [ ]:
df_metrics_felm = pd.concat([df_metrics_felm, df_metrics_felm_5layer])
df_metrics_factalign = pd.concat([df_metrics_factalign, df_metrics_factalign_5layer])
df_metrics_factbench = pd.concat([df_metrics_factbench, df_metrics_factbench_5layer])

In [ ]:
df_metrics_felm

,classifier,layer,accuracy
0,svm,-1,0.601770
1,svm,-4,0.592920
2,svm,-8,0.592920
3,svm,-16,0.584071
4,svm,all,0.601770
0,lr,-1,0.619469
1,lr,-4,0.637168
2,lr,-8,0.566372
3,lr,-16,0.557522
4,lr,all,0.601770


## **Saving Dataframes**

Saving dataframes with metrics on Google Drive

In [ ]:
saving_path = '/content/drive/MyDrive/Ceccarelli_MasterThesis&Internship/Master Thesis/Results'

In [ ]:
with open(saving_path + "/metrics_saplma_factbench_query_answer.csv", "w") as f:
    df_metrics_factbench.to_csv(f, index=False)

with open(saving_path + "/metrics_saplma_factalign_query_answer.csv", "w") as f:
    df_metrics_factalign.to_csv(f, index=False)

with open(saving_path + "/metrics_saplma_felm_query_answer.csv", "w") as f:
    df_metrics_felm.to_csv(f, index=False)